In [1]:
import sys

from os import getcwd
from os.path import dirname, abspath


sys.path.append(dirname((abspath(getcwd()))))

In [2]:
import pandas as pd

from sklearn.utils import shuffle

from school_budget.config import NUMERIC_COLUMNS, LABELS
from school_budget.data import multilabel_train_test_split


df = pd.read_csv("../datasets/TrainingData.csv", index_col=0)
df = shuffle(df)
# Create the new DataFrame: numeric_data_only
numeric_data_only = df[NUMERIC_COLUMNS].fillna(-1000)
# Get labels and convert to dummy variables: label_dummies
label_dummies = pd.get_dummies(df[LABELS])
# Create training and test sets
X_train, X_test, y_train, y_test = multilabel_train_test_split(
    numeric_data_only,
    label_dummies,
    size=0.2, 
    seed=123,
    min_count=5
)

In [3]:
from sklearn.linear_model import LogisticRegression
from sklearn.multiclass import OneVsRestClassifier

from joblib import dump

MODEL_FILE = "classifier-1.0.0.joblib"
# Instantiate the classifier: clf
clf = OneVsRestClassifier(LogisticRegression())
# Fit the classifier to the training data
clf.fit(X_train, y_train)
# Print the accuracy
print("Accuracy: {}".format(clf.score(X_test, y_test)))
# Persist the trained model
dump(clf, MODEL_FILE)

Accuracy: 0.0


['classifier-1.0.0.joblib']

## Step 2: Improving the Model

### What's in a token?

Now you will use combine_text_columns to convert all training text data in your DataFrame to a single vector that can be passed to the vectorizer object and made into a bag-of-words using the .fit_transform() method.

In [5]:
# Import the CountVectorizer
from sklearn.feature_extraction.text import CountVectorizer

from school_budget.features import combine_text_columns


# Create the basic token pattern
TOKENS_BASIC = '\\S+(?=\\s+)'
# Create the alphanumeric token pattern
TOKENS_ALPHANUMERIC = '[A-Za-z0-9]+(?=\\s+)'
# Instantiate basic CountVectorizer: vec_basic
vec_basic = CountVectorizer(token_pattern=TOKENS_BASIC)
# Instantiate alphanumeric CountVectorizer: vec_alphanumeric
vec_alphanumeric = CountVectorizer(token_pattern=TOKENS_ALPHANUMERIC)
# Create the text vector
text_vector = combine_text_columns(df)
pd.set_option('max_colwidth', 1200)
# Fit and transform vec_basic
vec_basic.fit_transform(text_vector)
# Print number of tokens of vec_basic
print("There are {} tokens in the dataset".format(len(vec_basic.get_feature_names())))
# Fit and transform vec_alphanumeric
vec_alphanumeric.fit_transform(text_vector)
# Print number of tokens of vec_alphanumeric
print("There are {} alpha-numeric tokens in the dataset".format(len(vec_alphanumeric.get_feature_names())))

/Users/slaily/.local/share/virtualenvs/school_budget-dhmayhNO/lib/python3.10/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


There are 4757 tokens in the dataset
There are 3284 alpha-numeric tokens in the dataset
